# Search Asset Vulnerability

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os, sys
import requests
import pandas as pd
from IPython.display import display

from datetime import datetime
nowTime = datetime.datetime.now().strftime("(%y%m%d-%H%M%S) ")

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# Aquasec URL
base_url_nonprod = os.getenv("AQUASEC_NONPROD_URL")
base_url_prod = os.getenv("AQUASEC_PROD_URL")
## Select URL
BASE_URL = base_url_nonprod

In [ ]:
from aquasec_authen import Authentication
# Credentials
username = os.getenv("AQUASEC_USER")
password = os.getenv("AQUASEC_PASSWORD")

# HEADER = Authentication(BASE_URL=BASE_URL).generateCredentialHeader(username=username, password=password)
HEADER = Authentication(BASE_URL=BASE_URL).generateTokenHeader(username=username, password=password)

HEADER

## Search Vulnerabilty of Asset by Asset ID

In [ ]:
def extractAssetVuln(assetID="", asset={}):
    return {
        "assetID": str(assetID),
        "name": asset["name"],
        "type": asset["resource"]["type"],
        "resource": asset["resource"]["name"],
        "version": asset["resource"]["version"],
        "fix_version": asset["fix_version"],
        "severity": asset["aqua_severity"],
        "status": "Need to update packages to a new version",
        "solution": asset["solution"],
        "description": asset["description"]
    }

Support Only first **200** vulnerabilities

In [ ]:
def searchAssetVulnbyID(assetID=""):
    try:
        print("\n[Request] Search Asset Vulnability by ID")
        url = BASE_URL + '/api/v1/hosts/' + \
            str(assetID) + '/vulnerabilities?include_vpatch_info=true&show_negligible=true&page=1&pagesize=10000&skip_count=true&hide_base_image=false&node_id=' + str(assetID)
        print("URL      : " + url)
        print("Asset ID : " + str(assetID))

        r = requests.get(url, headers=HEADER)
        if r.status_code == 200:
            r = r.json()
            print("\nGetting Successful")
            if r["result"] and r["count"] == len(r["result"]):
                print("Found Results " + str(len(r["result"])) + " of " + str(r["count"]))
                return [extractAssetVuln(assetID=assetID, asset=res) for res in r["result"]]
            else:
                print("[Error] Can't Search Asset Vulnability by ID")
                print("        Found Vulerabilities " + str(r["count"])  + ", Get :" + str(len(r["result"])) + " vulerabilities")
                sys.exit()
        else:
            print("\n[Error] " + str(r.status_code))
            print("        " + str(r.text))
            sys.exit()
    except:
        print("\n[Error] Function searchAssetVulnbyID")
        sys.exit()

In [ ]:
assetID = "<Asset ID | GUID>"

In [ ]:
assetVulnbyID = searchAssetVulnbyID(assetID=assetID)

In [ ]:
df = pd.DataFrame(assetVulnbyID)
len(df.index)

In [ ]:
data_severity = df.groupby('severity')

# severities = ["critical", "high", "medium", "low", "information"]
for severity in data_severity.groups:
    print(severity[0].upper() + severity[1:])
    assets = list(data_severity.groups[severity])
    print(len(assets), assets)
    display(df[df["severity"] == severity])
    filepath = "outputs/" + nowTime + assetID + "_" + severity
    df[df["severity"] == severity].to_csv(filepath + '.csv', index=False, encoding='utf-8')